In [2]:
import sys
from pathlib import Path

In [3]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [4]:
from api.db.session import engine
from api.events.models import EventModel
from sqlmodel import Session, select
from pprint import pprint
from timescaledb.hyperfunctions import time_bucket
from sqlalchemy import func

In [5]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.desc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT :param_1


In [13]:
from datetime import datetime, timedelta, timezone
with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket, 
            EventModel.page,
            func.count().label("event_count")
            )
        .where(
            # EventModel.time > start, 
            # EventModel.time <= finish,
            EventModel.page.in_(["/about", "pricing"])
        )
        .group_by(
            bucket,
            EventModel.page,
            )
        .order_by(
            bucket,
            EventModel.page
            )
        )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

SELECT time_bucket('1 day'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS event_count 
FROM eventmodel 
WHERE eventmodel.page IN ('/about', 'pricing') GROUP BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page
[(datetime.datetime(2025, 3, 26, 0, 0, tzinfo=datetime.timezone.utc), '/about', 2545),
 (datetime.datetime(2025, 3, 26, 0, 0, tzinfo=datetime.timezone.utc), 'pricing', 2504)]
